In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, LongType
from pyspark.sql.functions import split, col,substring,regexp_replace,length,expr,concat,concat_ws, max
import pyspark.sql.functions as f


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
# Importer tous les fichiers csv
df = spark.read.csv('/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/Data',header=True)

In [4]:
df.count()

7620638

In [5]:
#Traitement des colonnes !

#df = df.withColumn('LATITUDE', df['LATITUDE'].cast("float"))
df = df.withColumn('STATION', df['STATION'].cast("long"))
#df = df.withColumn('LONGITUDE', df['LONGITUDE'].cast("float"))
split_col = split(df['TMP'], ',')
df = df.withColumn('TMP_FINAL', (split_col.getItem(0).cast("int"))/10)
df = df.withColumn('TMP_CONTROLE', split_col.getItem(1))
split_col_date_heure = split(df['DATE'],'T')
df = df.withColumn('DATE_FINALE', split_col_date_heure.getItem(0))
df = df.withColumn('HEURE_FINALE', split_col_date_heure.getItem(1))
split_col_date = split(df['DATE_FINALE'],'-')
df = df.withColumn('JOUR', split_col_date.getItem(2).cast("int"))
df = df.withColumn('MOIS', split_col_date.getItem(1).cast("int"))
df = df.withColumn('ANNEE', split_col_date.getItem(0).cast("int"))
split_col_wnd = split(df['WND'],',')
df = df.withColumn('direction_angle', split_col_wnd.getItem(0).cast("int"))
df = df.withColumn('direction_quality_code', split_col_wnd.getItem(1).cast("int"))
df = df.withColumn('type_code', split_col_wnd.getItem(2))
df = df.withColumn('speed_rate', split_col_wnd.getItem(3).cast("int"))
df = df.withColumn('speed_quality_code', split_col_wnd.getItem(4).cast("int"))

In [6]:
# Selectionner les colonnes qu'on utilisera
df.select("NAME","STATION","JOUR","MOIS","ANNEE","TMP_FINAL","TMP_CONTROLE","direction_angle","direction_quality_code","type_code","speed_rate","speed_quality_code").show()

+-----------+----------+----+----+-----+---------+------------+---------------+----------------------+---------+----------+------------------+
|       NAME|   STATION|JOUR|MOIS|ANNEE|TMP_FINAL|TMP_CONTROLE|direction_angle|direction_quality_code|type_code|speed_rate|speed_quality_code|
+-----------+----------+----+----+-----+---------+------------+---------------+----------------------+---------+----------+------------------+
|SATENAS, SW|2520099999|   1|   1| 2021|      1.3|           1|            130|                     1|        N|        30|                 1|
|SATENAS, SW|2520099999|   1|   1| 2021|      1.0|           1|            150|                     1|        N|        26|                 1|
|SATENAS, SW|2520099999|   1|   1| 2021|      1.0|           1|            190|                     1|        V|        15|                 1|
|SATENAS, SW|2520099999|   1|   1| 2021|      0.8|           1|            200|                     1|        N|        10|                 1|

In [7]:
# Création d'un nouveau DataFrame avec les nouvelles colonnes à partir de DataFrame de base
df_f = df.select("NAME","STATION","LATITUDE","LONGITUDE","JOUR","MOIS","ANNEE","TMP_FINAL","TMP_CONTROLE","direction_angle","direction_quality_code","type_code","speed_rate","speed_quality_code")

In [8]:
df_f.count()

7620638

In [9]:
# Garder uniquement les lignes qui ont un TMP_CONROLE==1
df_f = df_f.filter(df_f.TMP_CONTROLE==1)

In [10]:
df_f.count()

7475333

In [11]:
df_f.show(n=1, truncate=False, vertical=True)

-RECORD 0-----------------------------
 NAME                   | SATENAS, SW 
 STATION                | 2520099999  
 LATITUDE               | 58.426445   
 LONGITUDE              | 12.714389   
 JOUR                   | 1           
 MOIS                   | 1           
 ANNEE                  | 2021        
 TMP_FINAL              | 1.3         
 TMP_CONTROLE           | 1           
 direction_angle        | 130         
 direction_quality_code | 1           
 type_code              | N           
 speed_rate             | 30          
 speed_quality_code     | 1           
only showing top 1 row



In [13]:
# MOYENNE DES TEMPÉRATURES PAR STATTION
df_f.groupby("LATITUDE").avg("TMP_FINAL").count()

366

In [14]:
# MOYENNE DES TEMPÉRATURES FINALES PAR LONGITUDE ET LATITUDE
df_f.groupby("STATION","NAME","LONGITUDE","LATITUDE").avg("TMP_FINAL").show(n=1, truncate=False, vertical=True)

-RECORD 0---------------------------
 STATION        | 2186099999        
 NAME           | KALLAX, SW        
 LONGITUDE      | 22.121989         
 LATITUDE       | 65.543758         
 avg(TMP_FINAL) | 4.038190265226524 
only showing top 1 row



In [15]:
# MOYENNE DES TEMPÉRATURES FINALES PAR LONGITUDE ET LATITUDE (STATION)
df_moyenne_station = df_f.groupby("STATION","NAME","LONGITUDE","LATITUDE",).avg("TMP_FINAL")
df_moyenne_station = df_moyenne_station.withColumnRenamed("avg(TMP_FINAL)","Moyenne_TMP_station")

In [16]:
#DataFrame moyenne des températures par station année 2020-2021
df_moyenne_station.show()

+----------+--------------------+----------+----------+-------------------+
|   STATION|                NAME| LONGITUDE|  LATITUDE|Moyenne_TMP_station|
+----------+--------------------+----------+----------+-------------------+
|2186099999|          KALLAX, SW| 22.121989| 65.543758|  4.038190265226524|
|1023099999|       BARDUFOSS, NO| 18.540356| 69.055758|  2.703143944144881|
|2520099999|         SATENAS, SW| 12.714389| 58.426445|   8.54774016254455|
|2458099999|         UPPSALA, SW| 17.588581| 59.897328|  7.632638943790031|
|2562099999|          MALMEN, SW| 15.525683| 58.402278| 7.4926822558459865|
|2154099999|          VIDSEL, SW| 20.149917| 65.875325| 2.6239300440779223|
|2526099999|      LANDVETTER, SW| 12.279819| 57.662836|  7.852673827673833|
|1311099999|        FLESLAND, NO|  5.218142| 60.293386|  8.364714129927082|
|2435099999|        BORLANGE, SW| 15.515211| 60.422017|  6.803027897701959|
|2101099999|         HEMAVAN, SW| 15.082778| 65.806111| 1.2369364813878323|
|2366099999|

In [17]:
df_moyenne_station.count()

448

In [18]:
# Exporter le résultat dans un fichier csv
#df_moyenne_station.write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/datacsv")

In [19]:
# MOYENNE DES TEMPÉRATURES FINALES PAR JOUR
df_moyenne_jour = df_f.groupby("STATION","NAME","LONGITUDE","LATITUDE",concat_ws('-',df_f.JOUR,df_f.MOIS,df_f.ANNEE)).avg("TMP_FINAL").orderBy("concat_ws(-, JOUR, MOIS, ANNEE)")
#df_moyenne_jour = df_moyenne_station.withColumnRenamed("concat_ws('-',df_f.JOUR,df_f.MOIS,df_f.ANNEE)","DATE")
#df_moyenne_jour = df_moyenne_station.withColumnRenamed("avg(TMP_FINAL)","Moyenne_TMP_jour")

In [20]:
df_moyenne_jour.show()

+----------+--------------------+----------+----------+-------------------------------+------------------+
|   STATION|                NAME| LONGITUDE|  LATITUDE|concat_ws(-, JOUR, MOIS, ANNEE)|    avg(TMP_FINAL)|
+----------+--------------------+----------+----------+-------------------------------+------------------+
|2431099999|          KLOTEN, SW|     15.25|59.8666666|                       1-1-2020|             2.375|
|2209099999|         SYLARNA, SW|12.2833333|     63.05|                       1-1-2020|0.7208333333333332|
|2265099999|         HEMLING, SW|     18.55|     63.65|                       1-1-2020|3.9583333333333335|
|2286099999|            UMEA, SW| 20.282758| 63.791828|                       1-1-2020| 2.204166666666667|
|2151099999|    JOKKMOKK FPL, SW|20.1666666|66.4833333|                       1-1-2020|-3.229166666666668|
|1384099999|      GARDERMOEN, NO| 11.100361| 60.193917|                       1-1-2020|1.9430555555555553|
|2489099999|           BERGA, SW|18.1

In [21]:
# Exporter le résultat dans un fichier csv
#df_moyenne_jour.write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/datacsvjour")

In [22]:
df_moyenne_jour.count()

285695

In [23]:
# Exporter le résultat dans un fichier csv
#df_moyenne_jour.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/datall")

In [24]:
# MOYENNE DES TEMPÉRATURES FINALES PAR MOIS
df_moyenne_mois = df_f.groupby("STATION","NAME","LONGITUDE","LATITUDE",concat_ws('-',df_f.MOIS,df_f.ANNEE)).avg("TMP_FINAL").orderBy("concat_ws(-, MOIS, ANNEE)")

In [25]:
df_moyenne_mois.count()

9819

In [26]:
# Exporter le résultat dans un fichier csv
#df_moyenne_mois.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/datalll")

In [27]:
# MOYENNE DES TEMPÉRATURES FINALES PAR ANNÉE
df_moyenne_annee = df_f.groupby("STATION","NAME","LONGITUDE","LATITUDE",concat_ws('-',df_f.ANNEE)).avg("TMP_FINAL").orderBy("concat_ws(-, ANNEE)")

In [28]:
df_moyenne_annee.show()

+----------+--------------------+---------+---------+-------------------+-------------------+
|   STATION|                NAME|LONGITUDE| LATITUDE|concat_ws(-, ANNEE)|     avg(TMP_FINAL)|
+----------+--------------------+---------+---------+-------------------+-------------------+
|2418099999|        KARLSTAD, SW|13.337361|59.444714|               2020|  8.445630018529943|
|1008099999|        LONGYEAR, SV|15.465556|78.246111|               2020|-3.4364999761188266|
|2049099999|       GALLIVARE, SW|20.814636|67.132408|               2020| 1.7815976495811157|
|2520099999|         SATENAS, SW|12.714389|58.426445|               2020|  9.261794225925193|
|2044099999|          KIRUNA, SW|20.336764|67.821986|               2020| 1.3389512532677403|
|1025099999|          TROMSO, NO|18.918919|69.683333|               2020|   4.12877777777781|
|1010099999|          ANDOYA, NO|16.144167|  69.2925|               2020|  5.141248704250044|
|2293099999|      SKELLEFTEA, SW|21.076892|64.624772|       

In [29]:
# Exporter le résultat dans un fichier csv
#df_moyenne_annee.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/dataannee")

In [30]:
# MOYENNE DES TEMPÉRATURES FINALES PAR PAYS
df_moyenne_pays = df_f.groupby(split(df['NAME'],',').getItem(1)).avg("TMP_FINAL")

In [31]:
df_moyenne_pays.show()

+---------------------+------------------+
|split(NAME, ,, -1)[1]|    avg(TMP_FINAL)|
+---------------------+------------------+
|                   SW|5.2510318377722145|
|                   NO|6.1057022527750835|
|                   SV|-5.192718321322931|
|                   FO| 9.968988594589499|
+---------------------+------------------+



In [32]:
# Exporter le résultat dans un fichier csv
#df_moyenne_pays.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/datapays")

In [33]:
# MOYENNE DES TEMPÉRATURES FINALES PAR PAYS ET PAR ANNÉE
df_moyenne_pays_annee = df_f.groupby(csplit(df['NAME'],',').getItem(1), concat_ws('-',df_f.ANNEE)).avg("TMP_FINAL")

In [34]:
df_moyenne_pays_annee.show()

+---------------------+-------------------+------------------+
|split(NAME, ,, -1)[1]|concat_ws(-, ANNEE)|    avg(TMP_FINAL)|
+---------------------+-------------------+------------------+
|                   SW|               2021| 4.983275450784886|
|                   NO|               2020| 6.464882753719924|
|                   SW|               2020| 5.608146716253506|
|                   SV|               2020|-5.255134528673344|
|                   NO|               2021| 5.601003277356129|
|                   FO|               2021|10.063603365781224|
|                   FO|               2020| 9.842897977132806|
|                   SV|               2021|-4.251914486279477|
+---------------------+-------------------+------------------+



In [35]:
# Exporter le résultat dans un fichier csv
#df_moyenne_pays_annee.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/datapaysannee")

In [36]:
# LA TEMPÉRATURE MAXIMALE PAR STATION
#df_MAX_station = df_f.groupby("STATION","NAME","LONGITUDE","LATITUDE",).max(df_f['TMP_FINAL'])


#df_MAX_station = df_f.groupby("LATITUDE").agg(f.max(df_f["TMP_FINAL"]))


df_MAX_station = df_f.groupby("STATION").agg(max(df_f['TMP_FINAL']))

In [37]:
df_MAX_station.show()

+----------+--------------+
|   STATION|max(TMP_FINAL)|
+----------+--------------+
|2154099999|          32.0|
|2520099999|          29.0|
|2562099999|          31.0|
|2186099999|          28.0|
|1023099999|          26.0|
|2458099999|          31.0|
|2366099999|          30.0|
|2590099999|          30.4|
|2526099999|          29.0|
|2267099999|          30.0|
|2101099999|          29.0|
|1311099999|          29.0|
|2435099999|          30.0|
|2418099999|          30.0|
|2563599999|          30.0|
|2044099999|          30.0|
|1271099999|          31.0|
|2049099999|          31.0|
|1025099999|          25.0|
|2293099999|          31.0|
+----------+--------------+
only showing top 20 rows



In [39]:
# Min Max Avg par station avec la requête SQL
df_f.createOrReplaceTempView("df_f_sql")
df_stattinmax=spark.sql("select STATION,NAME,LONGITUDE,LATITUDE, MAX(TMP_FINAL), MIN(TMP_FINAL), AVG(TMP_FINAL) from df_f_sql group by STATION,NAME,LONGITUDE,LATITUDE")

In [40]:
df_stattinmax.show()

+----------+--------------------+----------+----------+--------------+--------------+------------------+
|   STATION|                NAME| LONGITUDE|  LATITUDE|max(TMP_FINAL)|min(TMP_FINAL)|    avg(TMP_FINAL)|
+----------+--------------------+----------+----------+--------------+--------------+------------------+
|2186099999|          KALLAX, SW| 22.121989| 65.543758|          28.0|         -29.1| 4.038190265226524|
|1023099999|       BARDUFOSS, NO| 18.540356| 69.055758|          26.0|         -28.1| 2.703143944144881|
|2520099999|         SATENAS, SW| 12.714389| 58.426445|          29.0|         -16.1|  8.54774016254455|
|2458099999|         UPPSALA, SW| 17.588581| 59.897328|          31.0|         -25.0| 7.632638943790031|
|2562099999|          MALMEN, SW| 15.525683| 58.402278|          31.0|         -20.0|7.4926822558459865|
|2154099999|          VIDSEL, SW| 20.149917| 65.875325|          32.0|         -32.7|2.6239300440779223|
|2526099999|      LANDVETTER, SW| 12.279819| 57.662836|

In [41]:
df_stattinmax.count()

448

In [42]:
# Exporter le résultat dans un fichier csv
#df_stattinmax.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/dataMINMAX")

In [63]:
# Min Max Avg par station et par année
#df_f.createOrReplaceTempView("df_f_sql_annee")
df_stattinmax_annee=spark.sql("select STATION,NAME,LONGITUDE,LATITUDE,ANNEE, MAX(TMP_FINAL), MIN(TMP_FINAL), AVG(TMP_FINAL) from df_f_sql group by STATION,NAME,LONGITUDE,LATITUDE,ANNEE")

In [64]:
df_stattinmax_annee.show()

+----------+--------------------+----------+----------+-----+--------------+--------------+-------------------+
|   STATION|                NAME| LONGITUDE|  LATITUDE|ANNEE|max(TMP_FINAL)|min(TMP_FINAL)|     avg(TMP_FINAL)|
+----------+--------------------+----------+----------+-----+--------------+--------------+-------------------+
|2458099999|         UPPSALA, SW| 17.588581| 59.897328| 2020|          30.0|         -10.0|  8.422323968359137|
|2520099999|         SATENAS, SW| 12.714389| 58.426445| 2021|          29.0|         -16.1|  7.825258782936006|
|2186099999|          KALLAX, SW| 22.121989| 65.543758| 2020|          28.0|         -21.4|  5.064270967239313|
|1023099999|       BARDUFOSS, NO| 18.540356| 69.055758| 2020|          26.0|         -28.1|  2.703143944144881|
|2154099999|          VIDSEL, SW| 20.149917| 65.875325| 2020|          30.0|         -28.0| 3.6138797065226247|
|2520099999|         SATENAS, SW| 12.714389| 58.426445| 2020|          29.0|          -8.0|  9.261794225

In [65]:
df_stattinmax_annee.count()

829

In [56]:
# Exporter le résultat dans un fichier csv
#df_stattinmax_annee.coalesce(1).write.csv("/Users/mohamedsafir/Desktop/Ecole IPSSI/DataLake/dataMINMAX_annee")